In [20]:
import socket
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import Alignment

HOST = '127.0.0.1'  # The server's hostname or IP address
PORT = 12345  # The port number that the server listens on
EXCEL_FILENAME = 'output.xlsx'  # Name of the Excel file

# Create a TCP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to the server's address and port
server_socket.bind((HOST, PORT))

# Listen for incoming connections
server_socket.listen(1)
print('Server listening on', HOST, PORT)

# Accept an incoming connection
client_socket, client_address = server_socket.accept()
print('Connected by', client_address)

# Receive the operation from the client
operation = client_socket.recv(1024).decode()
print('Received operation:', operation)

# Perform the operation
result = eval(operation)
print('Result:', result)

# Send the result to the client
client_socket.sendall(str(result).encode())

# Write to Excel file
if not EXCEL_FILENAME:
    workbook = Workbook()
else:
    try:
        workbook = load_workbook(EXCEL_FILENAME)
    except FileNotFoundError:
        workbook = Workbook()

sheet = workbook.active

# Add headers if the sheet is empty
if sheet.max_row == 1 and sheet.max_column == 1 and not sheet.cell(1, 1).value:
    sheet.cell(1, 1, "Client")
    sheet.cell(1, 2, "Server")

# Append client and server data to Excel
row = [operation, result]
sheet.append(row)

# Set column width
column_width = 2 * max(len(str(operation)), len(str(result)))
column_letter = sheet.cell(1, 1).column_letter
sheet.column_dimensions[column_letter].width = column_width

column_letter = sheet.cell(1, 2).column_letter
sheet.column_dimensions[column_letter].width = column_width

# Set alignment for headers
header_alignment = Alignment(horizontal='center', vertical='center')
for cell in sheet[1]:
    cell.alignment = header_alignment

# Set alignment for data
data_alignment = Alignment(horizontal='center', vertical='center')
for row in sheet.iter_rows(min_row=2):
    for cell in row:
        cell.alignment = data_alignment

# Save the Excel file
workbook.save(EXCEL_FILENAME)

# Close the client socket
client_socket.close()

# Close the server socket
server_socket.close()

Server listening on 127.0.0.1 12345
Connected by ('127.0.0.1', 17467)
Received operation: 7 * 2
Result: 14
